## Vault Installation to Minikube via Helm with Integrated Storage

### Start Minikube
- we use minikube to create k8s cluster on your local machine
- start k8s with minikube with command: minikube start
- check cluster status with command: minikube status
- reference link: https://developer.hashicorp.com/vault/tutorials/kubernetes/kubernetes-minikube-raft

### Install the Vault Helm chart
- we use helm chart to install vault
- with helm we have to understand the concept of chart and repo
    - chart: is a collection of k8s manifest which we want to deploy together
    - repo: is a collection of chart
- add repo to helm: 
    - helm repo add $repo_name $repo_address
- update all repo in local to synx with remote repo: 
    - helm repo update
- check chart have been add with command: 
    - helm search repo $chart_name
- write file content to a file using cat command:
    - cat > $file_name << EOF file content ... EOF
- install a chart with helm install command:
    - helm install $release_name $chart_name --values $path_to_config_file.yaml
- list pods in kubectl get command
    - kubectl get pods
- generate root key by using --vault command inside a container
    - run command inside container kubectl exec
        - kubectl exec $pod_name -- $command
    - example:
        - kubectl exec vault-0 -- vault operator init \
            -key-shares=1 \
            -key-threshold=1 \
            -format=json > cluster-keys.json
        - command meaning: run vault operator init command to create root key inside the pod
- read json file with jq cli:
    - jq cli is command line tools which deal with json file
    - jq -r ".unseal_keys_b64[]" cluster-keys.json
- create environment variable to store key
    - VAULT_UNSEAL_KEY=$(jq -r ".unseal_keys_b64[]" cluster-keys.json)
    - to capture return value and then assign to variable
        - variable_name=$(command)
- unseal vault
    - kubectl exec vault-0 -- vault operator unseal $VAULT_UNSEAL_KEY
- add vault-1, vault-2 to raft cluster
    - kubectl exec -ti vault-1 -- vault operator raft join http://vault-0.vault-internal:8200
    - kubectl exec -ti vault-2 -- vault operator raft join http://vault-0.vault-internal:8200
- unseal vault-1, vault-2
    - kubectl exec -ti vault-1 -- vault operator unseal $VAULT_UNSEAL_KEY
    - kubectl exec -ti vault-2 -- vault operator unseal $VAULT_UNSEAL_KEY

### Set a secret in Vault
- display root key:
    - jq -r ".root_token" cluster-keys.json
- start interactive shell section on pod vault-0
    - kubectl exec --stdin=true --tty=true vault-0 -- /bin/sh
- login with root token
    - vault login
- enable kv-v2 secret engine
    - kv-v2 secret engine which allow us to create and manage key value type secret in vault
    - vault secrets enable -path=secret kv-v2
- create secret
    - vault kv put
    - example:
        - vault kv put secret/webapp/config username="static-user" password="static-password"
- not about secrets:
    - secret name as path
        - example: secret/webapp/config
    - secret path act like an id to reference to that secret from client
    - metadata: support information about secret
    - data: key, value pair content of secret
- check secret information
    - vault kv get
    - example:
        - vault kv get secret/webapp/config
- exit interactive shell session on pd
    - exit

### Configure Kubernetes authentication
- this part is how to set authentication which allow application to access secret have been put to vault
- start interactive shell
    - kubectl exec --stdin=true --tty=true vault-0 -- /bin/sh
- enable k8s authenticate in vault
    - vault auth enable kubernetes
- config k8s authentication method
    - vault write auth/kubernetes/config \
    kubernetes_host="https://$KUBERNETES_PORT_443_TCP_ADDR:443"
    - KUBERNETES_PORT_443_TCP_ADDR is k8s api host address
- write policy
    - vault policy write webapp - <<EOF
path "secret/data/webapp/config" {
  capabilities = ["read"]
}
EOF
    - policy is an object with allow capability action (read, write, etc) on secret given secret path
- create or write role
    - vault policy write webapp - <<EOF
path "secret/data/webapp/config" {
  capabilities = ["read"]
}
EOF
    - role is an object which allow bindinb between k8s service account and vault policy. Or this k8s service account can use this vault policy to access secret

- apply k8s Deployment
    - kubectl apply --filename deployment-01-webapp.yml
    - Deployment is an object which belong to WorkLoad object class
    - Deployment control pods deployment of stateless application
    - Deployment use other object for the pod deployment like: ConifgMa, Secret, ServiceAccount, Service
- forward request from host machine into container
    - kubectl port-forward \
    $(kubectl get pod -l app=webapp -o jsonpath="{.items[0].metadata.name}") \
    8080:8080
    - example:
        - kubectl port-forward pod/mypod 8888:5000
        - command above means listen on port 8888 locally and forward to container port 5000




### Summary
- generate root key --> use root key to login to vault
- generate secret command: vault kv put
- check secret with command: vault kv get
- policy is object about permission to a secret given it's path
- role is object biding k8s service account and poclicy